Морское сражение ботов на поле 4x4

Тип работы:  Индивидуальная работа

УРА! Настало время проверить вашего бота - насколько он умеет выбирать ходы.

Для этого надо объединить две ваши функции в одном скрипте. 

1. Функция, которая расставляет корабли (см. 3.15. Расстановка кораблей для морского боя)

`start_boats_position(how_many_boats)`

**ВХОДНЫЕ ДАННЫЕ**: `how_many_boats` - кортеж вида `(n1, n2)`, 1 <= n1 <= 3; 0 <= n2 <= 1

**ВЫХОД**: список, который содержит расстановку кораблей n1 однопалубных кораблей и n2 двухпалубных (если n2=1) в виде:

`[ [('a',1)], [('d',2)], [('b',1),('c',1)] ]`

если n1=2, n2=1.

2. Функция, которая выбирает ход (см. 4.16. Выбираем ход в морском бое)

`choose_act(battle_state, last_move, result):`

**ВХОДНЫЕ ДАННЫЕ**: `battle_state` - это двухмерный массив numpy 4х4, который содержит результат наших выстрелов.

**last_move** - последний сделанный вами ход в виде кортежа (letter, number), letter - это одна из букв a,b,c,d, а number - это целое число от 1 до 4. (см. 3.15. Расстановка кораблей для морского боя);

**result** - результат последнего хода в виде строки 'miss' | 'injury' | 'killed' (промах, ранен, убит) - см. 3.16. Определяем результат хода.

**ВЫХОД**: move = (letter,number)

ВАЖНОЕ ЗАМЕЧАНИЕ: Чтобы вам было легче добивать двухпалубные корабли после промаха, мы изменяем состояние ранен (battle_state[i,j]=1) на состояние 2 (убит), когда вы добьете двухпалубный корабль. Это сделано для того, чтобы вы могли быстро проверить - есть ли недобитый корабль по состоянию боя battle_state и добить его.

In [9]:
''' --------- модуль слушателя курса -------- '''

# функция начальной расстановки кораблей 1-палубных и 2-х палубных
import numpy as np
from numpy import random as rnd

def check_neighbors_init(t, bp):
    # returns True is there are no ships on the left, right, up or down
    # diagonal neighbors are allowed
    # `t` is proposed new position
    # `bp`` is flattened current boats_position

    for b in bp:
        # diagonal neighbor case
        if (abs(t[0] - b[0]) == 1) and (abs(t[1] - b[1]) == 1):
            continue
        if (abs(t[0] - b[0]) < 2) and (abs(t[1] - b[1]) < 2):
            return False

    return True

def flattenbp(bp):
    # flattens boats_position
    # i.e. [[(2, 3)], [(1, 2), (2, 2)]] -> [(2, 3), (1, 2), (2, 2)]
    res = []

    for b in bp:
        for t in b:
            res.append(t)

    return res

def replacecolswithletters(bp):
    # replace 1 with 'a',
    # 2 with 'b', etc.
    res = []
    cols = [*'_abcd']

    for i, b in enumerate(bp):
        res.append([])
        for t in b:
            res[i].append((cols[t[0]], t[1]))
    return res

def start_boats_position(how_many_boats):
    n1 = how_many_boats[0] # кол-во однопалубных
    n2 = how_many_boats[1] # кол-во двухпалубных

    boats_position = []

    if not (1 <= n1 <= 3 and 0 <= n2 <= 1):
        print("Количество 1-палубных кораблей: от 1 до 3\nКоличество 2-палубных кораблей: от 0 до 1")
        return

    if n2:
        r = rnd.randint(1, 5)
        c = rnd.randint(1, 5)
        lst = [(c, r)]

        while True:
            direction = rnd.randint(1, 5)
            # 1 - up, 2 - right, 3 - down, 4 - left
            if direction == 1 and r != 1:
                r -= 1
                break
            elif direction == 2 and c != 4:
                c += 1
                break
            elif direction == 3 and r != 4:
                r += 1
                break
            elif direction == 4 and c != 1:
                c -= 1
                break
        lst.append((c, r))
        boats_position.append(lst)

    for _ in range(n1):
        flattenedbp = flattenbp(boats_position)
        while True:
            r = rnd.randint(1, 5)
            c = rnd.randint(1, 5)
            if check_neighbors_init((c, r), flattenedbp):
                boats_position.append([(c, r)])
                break

    boats_position = replacecolswithletters(boats_position)
    return boats_position



def is_2_deck_ship_dead(battle_state):
    for i in range(battle_state.shape[0]):
        for j in range(battle_state.shape[1]):
            if battle_state[i, j] == 2:
                if i > 0:   # up
                    if battle_state[i - 1, j] == 2:
                        return True
                if j < 3:   # right
                    if battle_state[i, j + 1] == 2:
                        return True
                if i < 3:   # down
                    if battle_state[i + 1, j] == 2:
                        return True
                if j > 0:   # left
                    if battle_state[i, j - 1] == 2:
                        return True
    return False

def kill_boat(battle_state, last_move):
    '''
    :param battle_state: двухмерный массив numpy 4х4, который содержит результат наших выстрелов
    battle_state[i,j] < 0, то стреляли по полю (i,j) и ничего, 
    battle_state[i,j] == 0, то не стреляли; если 1 или 2 - ранили и убили
    :param last_move: последний ваш ход (letter, number),который ранил 2-х палубный 
    :return: move=(letter, number) - выбранный ход, чтобы добить корабль
    '''
    move = ("z", 0) # пустой ход

    cti = str.maketrans({'a': '1', 'b': '2', 'c': '3', 'd': '4' })
    last_move_i = (int(last_move[0].translate(cti)), last_move[1])
    itc = "abcd"

    move_candidates = []
    # up
    if (last_move_i[1] - 1) > 0:
        move_candidates.append(tuple((last_move_i[0], last_move_i[1] - 1)))
    # right
    if (last_move_i[0] + 1) < 5:
        move_candidates.append(tuple((last_move_i[0] + 1, last_move_i[1])))
    # down
    if (last_move_i[1] + 1) < 5:
        move_candidates.append(tuple((last_move_i[0], last_move_i[1] + 1)))
    # left
    if (last_move_i[0] - 1) > 0:
        move_candidates.append(tuple((last_move_i[0] - 1, last_move_i[1])))

    for mc in move_candidates:
        # clean coordinates: [1-4] -> [0-3]
        # also reversed, because map coordinates (move, last_move) are in the form [column, row]
        # and battle_state is in the form [row, column]
        coords = (mc[1]-1, mc[0]-1)

        if battle_state[coords[0], coords[1]] != 0:
            continue
        if (coords[0]-1) >= 0:
            if battle_state[coords[0]-1, coords[1]] == 2:
                continue
        if (coords[1]+1) <= 3:
            if battle_state[coords[0], coords[1]+1] == 2:
                continue
        if (coords[0]+1) <= 3:
            if battle_state[coords[0]+1, coords[1]] == 2:
                continue
        if (coords[1]-1) >= 0:
            if battle_state[coords[0], coords[1]-1] == 2:
                continue
        move = (itc[coords[1]], mc[1])
        break

    return move


def check_neighbors(move, battle_state):
    if move[0] > 0: # up
        if battle_state[move[0] - 1, move[1]] == 2:
            return False
    if move[1] < 3: # right
        if battle_state[move[0], move[1] + 1] == 2:
            return False
    if move[0] < 3: # down
        if battle_state[move[0] + 1, move[1]] == 2:
            return False
    if move[1] > 0: # left
        if battle_state[move[0], move[1] - 1] == 2:
            return False
    return True


# ---------- функция выбора нового хода ----------
def choose_act(battle_state, last_move, result):
    # battle_state - это двухмерный массив numpy 4х4, который содержит результат наших выстрелов
    # last_move - последний сделанный вами ход в виде кортежа (letter, number),
    # letter - это одна из букв a,b,c,d, а number - это целое число от 1 до 4. (см. 3.15. Расстановка кораблей для морского боя);
    # result - результат последнего хода в виде строки 'miss' | 'injury' | 'killed' (промах, ранен, убит) - см. 3.16. Определяем результат хода.

    move = ("z", 0)
    new_move_is_determined = False

    # finish off 2-deck ship
    for i in range(4):
        for j in range(4):
            if battle_state[i, j] == 1:
                move = kill_boat(battle_state, ('abcd'[j], i + 1))
                return move

    two_deck_ship_is_dead = is_2_deck_ship_dead(battle_state)

    for i in range(4):
        if new_move_is_determined:
            break
        for j in range(4):
            if battle_state[i, j] != 0:
                continue
            # check self
            if not check_neighbors((i, j), battle_state):
                continue
            if two_deck_ship_is_dead:   # search for 1-deck ship
                move = ('abcd'[j], i + 1)
                return move
            else:                       # search for 2-deck ship
                # check possible second decks
                if i > 0:   # up
                    if battle_state[i - 1, j] == 0 and check_neighbors((i - 1, j), battle_state):
                        move = ('abcd'[j], i + 1)
                        new_move_is_determined = True
                        break
                if j < 3:   # right
                    if battle_state[i, j + 1] == 0 and check_neighbors((i, j + 1), battle_state):
                        move = ('abcd'[j], i + 1)
                        new_move_is_determined = True
                        break
                if i < 3:   # down
                    if battle_state[i + 1, j] == 0 and check_neighbors((i + 1, j), battle_state):
                        move = ('abcd'[j], i + 1)
                        new_move_is_determined = True
                        break
                if j > 0:   # left
                    if battle_state[i, j - 1] == 0 and check_neighbors((i, j - 1), battle_state):
                        move = ('abcd'[j], i + 1)
                        new_move_is_determined = True
                        break
    return move


# start_boats_position(how_many_boats=(3,1))


# battle_state = np.array([[-1,-1,2,0],[0,0,0,0],[0,2,0,0],[2,0,0,0]])
# last_move = ('c', 1)
# result = 'killed'
# ('d', 2)

# battle_state = np.array([[0,-1,0,0], [0,0,2,0], [2,0,0,0], [0,1,0,-1]])
# last_move = ('d', 4)
# result = 'miss'
# ('c', 4)

# battle_state = np.array([[0,0,2,-1], [2,0,2,0], [0,-1,0,0], [-1,0,0,2]])
# last_move = ('c', 2)
# result = 'kill'

# choose_act(battle_state, last_move, result)

[[('a', 1), ('b', 1)], [('d', 1)], [('a', 3)], [('b', 4)]]